In [29]:
import os

import numpy as np
import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine

Завдання 3

In [30]:
load_dotenv()

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")


In [31]:
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

conn = engine.connect()

changelogs = pd.read_sql(
    "SELECT * FROM changelogs", conn
)
issues = pd.read_sql(
    "SELECT * FROM issues", conn
)
status_duration = pd.read_sql(
    "SELECT * FROM status_duration", conn
)

conn.close()

In [32]:
status_duration

,issue_key,author_key,status,start_date,end_date,calendar_duration
0,PYTHON-109,mike,Open,2010-03-05 18:51:36,2010-03-10 12:13:11,408095.0
1,PYTHON-108,mike,Open,2010-03-05 14:28:32,2010-08-19 15:58:33,14434201.0
2,PYTHON-107,mike,Open,2010-03-01 04:07:10,2010-03-02 12:08:58,115308.0
3,PYTHON-106,mike,Open,2010-02-28 19:54:09,2010-05-10 17:41:21,6126432.0
4,PYTHON-105,mike,Open,2010-02-19 22:54:44,2010-02-22 15:45:32,233448.0
...,...,...,...,...,...,...
8914,PYTHON-4,behackett,Open,2009-04-09 09:40:32,2011-07-12 02:47:50,71168838.0
8915,PYTHON-4,behackett,Resolved,2009-04-09 09:40:32,2011-08-05 23:04:33,73315441.0
8916,PYTHON-3,mike,Open,2009-04-09 09:39:56,2009-10-26 12:08:50,17288934.0
8917,PYTHON-2,mike,Open,2009-04-09 09:39:06,2009-07-20 17:19:39,8840433.0


In [33]:
status_duration["working_duration"] = np.nan


In [34]:
status_duration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8919 entries, 0 to 8918
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   issue_key          8919 non-null   object        
 1   author_key         8918 non-null   object        
 2   status             8919 non-null   object        
 3   start_date         8919 non-null   datetime64[ns]
 4   end_date           8919 non-null   datetime64[ns]
 5   calendar_duration  8919 non-null   float64       
 6   working_duration   0 non-null      float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 487.9+ KB


In [35]:
from datetime import timedelta, datetime


def calculate_working_seconds(start_date: datetime, end_date: datetime) -> int:
    current_date = start_date
    working_hour_start = 10
    working_hour_end = 20
    total_seconds = 0

    while current_date <= end_date:
        if current_date.weekday() not in (5, 6):
            start_work = current_date.replace(hour=working_hour_start, minute=0, second=0, microsecond=0)
            end_work = current_date.replace(hour=working_hour_end, minute=0, second=0, microsecond=0)

            if current_date.hour < working_hour_end:
                if current_date.hour >= working_hour_start:
                    if current_date.date() == end_date.date() and end_date.hour < working_hour_end:
                        total_seconds += (end_date - current_date).total_seconds()
                    else:
                        total_seconds += (end_work - current_date).total_seconds()
                else:
                    total_seconds += (end_work - start_work).total_seconds()

        current_date += timedelta(days=1)
        current_date = current_date.replace(hour=working_hour_start, minute=0, second=0, microsecond=0)

    return total_seconds


In [36]:
status_duration["working_duration"] = np.where(
    status_duration["status"] == "In Progress", status_duration.apply(
        lambda row: calculate_working_seconds(row["start_date"], row["end_date"]), axis=1),
    np.nan
)

status_duration

,issue_key,author_key,status,start_date,end_date,calendar_duration,working_duration
0,PYTHON-109,mike,Open,2010-03-05 18:51:36,2010-03-10 12:13:11,408095.0,NaN
1,PYTHON-108,mike,Open,2010-03-05 14:28:32,2010-08-19 15:58:33,14434201.0,NaN
2,PYTHON-107,mike,Open,2010-03-01 04:07:10,2010-03-02 12:08:58,115308.0,NaN
3,PYTHON-106,mike,Open,2010-02-28 19:54:09,2010-05-10 17:41:21,6126432.0,NaN
4,PYTHON-105,mike,Open,2010-02-19 22:54:44,2010-02-22 15:45:32,233448.0,NaN
...,...,...,...,...,...,...,...
8914,PYTHON-4,behackett,Open,2009-04-09 09:40:32,2011-07-12 02:47:50,71168838.0,NaN
8915,PYTHON-4,behackett,Resolved,2009-04-09 09:40:32,2011-08-05 23:04:33,73315441.0,NaN
8916,PYTHON-3,mike,Open,2009-04-09 09:39:56,2009-10-26 12:08:50,17288934.0,NaN
8917,PYTHON-2,mike,Open,2009-04-09 09:39:06,2009-07-20 17:19:39,8840433.0,NaN


Завдання 4
Якщо людина працює над декількома задачами одночасно - робочий час за цей період (коли він працює над декількома задачами) ділиться на кількість задач, над якими він працює.

In [37]:
status_duration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8919 entries, 0 to 8918
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   issue_key          8919 non-null   object        
 1   author_key         8918 non-null   object        
 2   status             8919 non-null   object        
 3   start_date         8919 non-null   datetime64[ns]
 4   end_date           8919 non-null   datetime64[ns]
 5   calendar_duration  8919 non-null   float64       
 6   working_duration   1378 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 487.9+ KB


In [38]:
df = status_duration


def distribute_working_hours(df: pd.DataFrame):
    df["working_duration"] = 0.0
    df_in_progress = df[df["status"] == "In Progress"]
    grouped = df_in_progress.groupby("author_key")

    for author, group in grouped:
        events = []
        for i, row in group.iterrows():
            events.append((row["start_date"], "start", row["issue_key"]))
            events.append((row["end_date"], "end", row["issue_key"]))

        events.sort()

        current_tasks = set()
        last_time = None

        for current_time, event_type, issue_key in events:
            if current_tasks and (event_type == "start" or event_type == "end"):
                split_working_duration = calculate_working_seconds(last_time, current_time) / len(current_tasks)

                for task in current_tasks:
                    condition = (df["issue_key"] == task) & (df["status"] == "In Progress")
                    df.loc[condition, "working_duration"] += split_working_duration

            if event_type == "start":
                current_tasks.add(issue_key)
            elif event_type == "end":
                if issue_key in current_tasks:
                    current_tasks.remove(issue_key)

            last_time = current_time

    return df

In [39]:
result_df = distribute_working_hours(df)
result_df

,issue_key,author_key,status,start_date,end_date,calendar_duration,working_duration
0,PYTHON-109,mike,Open,2010-03-05 18:51:36,2010-03-10 12:13:11,408095.0,0.0
1,PYTHON-108,mike,Open,2010-03-05 14:28:32,2010-08-19 15:58:33,14434201.0,0.0
2,PYTHON-107,mike,Open,2010-03-01 04:07:10,2010-03-02 12:08:58,115308.0,0.0
3,PYTHON-106,mike,Open,2010-02-28 19:54:09,2010-05-10 17:41:21,6126432.0,0.0
4,PYTHON-105,mike,Open,2010-02-19 22:54:44,2010-02-22 15:45:32,233448.0,0.0
...,...,...,...,...,...,...,...
8914,PYTHON-4,behackett,Open,2009-04-09 09:40:32,2011-07-12 02:47:50,71168838.0,0.0
8915,PYTHON-4,behackett,Resolved,2009-04-09 09:40:32,2011-08-05 23:04:33,73315441.0,0.0
8916,PYTHON-3,mike,Open,2009-04-09 09:39:56,2009-10-26 12:08:50,17288934.0,0.0
8917,PYTHON-2,mike,Open,2009-04-09 09:39:06,2009-07-20 17:19:39,8840433.0,0.0


Завдання 5
Розрахувати загальну вартість всіх задач в межах типу задач. Тип задач знаходиться в таблиці issues колонка issue_type. Розрахувати за такою логікою:

Якщо author_key починається з "JIRAUSER" - ціна години часу такого спеціаліста $1.00
Якщо author_key НЕ починається з "JIRAUSER" - ціна години часу такого спеціаліста $2.00
Результат:
Таблиця з полями - issue_type, total_cost

In [40]:
def calculate_cost(row: pd.Series):
    if row["author_key"]:
        price = 1.0 if row["author_key"].startswith("JIRAUSER") else 2.0
        cost = price * (row["working_duration"] / 3600)
        return cost
    return 0

In [41]:
result_df["task_cost"] = result_df.apply(calculate_cost, axis=1)
result_df

,issue_key,author_key,status,start_date,end_date,calendar_duration,working_duration,task_cost
0,PYTHON-109,mike,Open,2010-03-05 18:51:36,2010-03-10 12:13:11,408095.0,0.0,0.0
1,PYTHON-108,mike,Open,2010-03-05 14:28:32,2010-08-19 15:58:33,14434201.0,0.0,0.0
2,PYTHON-107,mike,Open,2010-03-01 04:07:10,2010-03-02 12:08:58,115308.0,0.0,0.0
3,PYTHON-106,mike,Open,2010-02-28 19:54:09,2010-05-10 17:41:21,6126432.0,0.0,0.0
4,PYTHON-105,mike,Open,2010-02-19 22:54:44,2010-02-22 15:45:32,233448.0,0.0,0.0
...,...,...,...,...,...,...,...,...
8914,PYTHON-4,behackett,Open,2009-04-09 09:40:32,2011-07-12 02:47:50,71168838.0,0.0,0.0
8915,PYTHON-4,behackett,Resolved,2009-04-09 09:40:32,2011-08-05 23:04:33,73315441.0,0.0,0.0
8916,PYTHON-3,mike,Open,2009-04-09 09:39:56,2009-10-26 12:08:50,17288934.0,0.0,0.0
8917,PYTHON-2,mike,Open,2009-04-09 09:39:06,2009-07-20 17:19:39,8840433.0,0.0,0.0


In [42]:
merged_df = pd.merge(result_df, issues[["issue_key", "issue_type"]], on="issue_key")
merged_df


,issue_key,author_key,status,start_date,end_date,calendar_duration,working_duration,task_cost,issue_type
0,PYTHON-109,mike,Open,2010-03-05 18:51:36,2010-03-10 12:13:11,408095.0,0.0,0.0,Improvement
1,PYTHON-108,mike,Open,2010-03-05 14:28:32,2010-08-19 15:58:33,14434201.0,0.0,0.0,Improvement
2,PYTHON-107,mike,Open,2010-03-01 04:07:10,2010-03-02 12:08:58,115308.0,0.0,0.0,Improvement
3,PYTHON-106,mike,Open,2010-02-28 19:54:09,2010-05-10 17:41:21,6126432.0,0.0,0.0,Bug
4,PYTHON-105,mike,Open,2010-02-19 22:54:44,2010-02-22 15:45:32,233448.0,0.0,0.0,Bug
...,...,...,...,...,...,...,...,...,...
8914,PYTHON-4,behackett,Open,2009-04-09 09:40:32,2011-07-12 02:47:50,71168838.0,0.0,0.0,Bug
8915,PYTHON-4,behackett,Resolved,2009-04-09 09:40:32,2011-08-05 23:04:33,73315441.0,0.0,0.0,Bug
8916,PYTHON-3,mike,Open,2009-04-09 09:39:56,2009-10-26 12:08:50,17288934.0,0.0,0.0,New Feature
8917,PYTHON-2,mike,Open,2009-04-09 09:39:06,2009-07-20 17:19:39,8840433.0,0.0,0.0,New Feature


In [43]:
total_cost_by_issue_type = merged_df.groupby("issue_type")["task_cost"].sum().reset_index()
total_cost_by_issue_type.columns = ["issue_type", "total_cost"]

total_cost_by_issue_type

,issue_type,total_cost
0,Bug,44528.027277
1,Epic,45629.985472
2,Improvement,115735.931810
3,New Feature,70215.913677
4,Question,0.000000
5,Spec Change,874.577644
6,Sub-task,112.959948
7,Task,102077.616847


extra: load in db

In [44]:
with engine.connect() as conn:
    total_cost_by_issue_type.to_sql("issue_costs", con=conn, if_exists="replace", index=False)


In [46]:
result_df.to_csv("result.csv")